In [1]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

from pathlib import Path
import pathlib
import pandas as pd
from data_loader import Dataloader_trdp

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
chip_dimension = 256

In [12]:
train_dir = Path('../DATASET/dataset_pruebas/train')
test_dir = Path('../DATASET/dataset_pruebas/validation')
sample_dir = Path('../DATASET/SN7_buildings_train_sample')

root_dir  = train_dir
csv_file = Path('../output_csvs/df_train_untidy.csv')
csv_file_test = Path('../output_csvs/df_test_untidy.csv')
df = pd.read_csv(csv_file)
df_test = pd.read_csv(csv_file_test)


In [13]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
A.Rotate(limit=(-360, 360), interpolation=4, border_mode=4,p=0),

chip_dimension = 256

transform = A.Compose(
    [
        A.PadIfNeeded(min_height=chip_dimension,min_width=chip_dimension,value=0,p=1),
        A.RandomRotate90(p=1.0),
        ToTensorV2()
    ]
)

In [14]:
train_set   = Dataloader_trdp(root_dir=root_dir,csv_file=csv_file,chip_dimension=chip_dimension,transform=transform)

testing_set = Dataloader_trdp(root_dir=root_dir,csv_file=csv_file_test,chip_dimension=chip_dimension,transform=transform)

In [15]:
print(f"len(train_set): {len(train_set)}")
#print(f"len(testing_set): {len(testing_set)}")

for i in range(len(train_set)):
    sample = train_set[i]
    # verify dimensionality
    print(sample['raster_diff'].shape, sample['mask_diff'].shape)
    if i==15:
        break

len(train_set): 1504
len(testing_set): 0
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.Size([3, 256, 256]) torch.Size([1, 256, 256])


In [16]:
train_set, test_set = torch.utils.data.random_split(train_set, [1000,504])

In [17]:
train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle = True)

In [18]:
# Model
model = torchvision.models.resnet50(pretrained=True)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)